In [ ]:
# fastai 0.7.2
!pip install git+https://github.com/fastai/fastai@e85667cfae2e6873b1bb026195b5d09a74dfcff9
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

from tqdm import tqdm

In [ ]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000, "display.precision", 15): 
        display(df)

In [ ]:
%%time
df_raw = pd.read_csv("../input/train.csv",
                     dtype={"acoustic_data": np.int16, "time_to_failure": np.float64},
                     nrows=100_000_000)

In [ ]:
display_all(df_raw.head(10).T)

In [ ]:
df_raw.describe()

In [ ]:
df_raw_f = df_raw[:100_000].copy()
stepsize = np.diff(df_raw_f.acoustic_data)
df_raw_f = df_raw_f.drop(df_raw_f.index[0])
df_raw_f["delta"] = stepsize


In [ ]:
df_raw_f.head()

In [ ]:
def create_train(raw_size, rows):
    df_raw_f = df_raw[:raw_size].copy()
    stepsize = np.diff(df_raw_f.acoustic_data)
    df_raw_f = df_raw_f.drop(df_raw_f.index[0])
    df_raw_f["delta"] = stepsize
    
    segments = int(np.floor(raw_size / rows))
    df_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                           columns=['ave', 'std', 'max', 'min',
                                    'stepsize_ave', 'stepsize_std', 'stepsize_max', 'stepsize_min',
                                    'time_to_failure'])
    for segment in tqdm(range(segments)):
        seg = df_raw_f.iloc[segment*rows:segment*rows+rows]
        # Take the latest because 
        # For each seg_id in the test folder, you should predict a single time_to_failure corresponding to the time between 
        # the last row of the segment and the next laboratory earthquake.
        y = seg['time_to_failure'].values[-1]

        df_train.loc[segment, 'ave'] = seg['acoustic_data'].values.mean()
        df_train.loc[segment, 'std'] = seg['acoustic_data'].values.std()
        df_train.loc[segment, 'max'] = seg['acoustic_data'].values.max()
        df_train.loc[segment, 'min'] = seg['acoustic_data'].values.min()
        df_train.loc[segment, 'stepsize_ave'] = seg['delta'].values.mean()
        df_train.loc[segment, 'stepsize_std'] = seg['delta'].values.std()
        df_train.loc[segment, 'stepsize_max'] = seg['delta'].values.max()
        df_train.loc[segment, 'stepsize_min'] = seg['delta'].values.min()
        df_train.loc[segment, 'time_to_failure'] = y
    return df_train

In [ ]:
df_train = create_train(500_000, 100);

In [ ]:
# df_train = df_train.sample(frac=1, axis=1).reset_index(drop=True)

In [ ]:
display_all(df_train.tail().T)

In [ ]:
df_test = pd.read_csv("../input/test/seg_04cceb.csv")
df_test.head()

# Make it stupid

In [ ]:
df_trn, y_trn, nas = proc_df(df_train, 'time_to_failure')

In [ ]:
def split_vals(a,n): 
    return a[:n].copy(), a[n:].copy()
    
train_required_ratio = 0.80
n_trn = int(len(df_trn) * train_required_ratio)

X_train, X_valid = split_vals(df_trn, n_trn)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
y_train, y_valid = split_vals(y_trn, n_trn)
X_train.shape, X_valid.shape

In [ ]:
def print_score(m):
    res = [metrics.mean_absolute_error(m.predict(X_train), y_train), metrics.mean_absolute_error(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [ ]:
m = RandomForestRegressor(n_estimators=100, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
# preds = np.stack([t.predict(X_valid) for t in m.estimators_])
# preds[:,0], np.mean(preds[:,0]), y_valid[0]

In [ ]:
# plt.plot([metrics.r2_score(y_valid, np.mean(preds[:i+1], axis=0)) for i in range(100)]);

In [ ]:
set_rf_samples(50_000)

In [ ]:
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.5,
                          n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
reset_rf_samples()

In [ ]:
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=2, max_features=1,
                          n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

Stupid approach doesn't work

In [ ]:
fi = rf_feat_importance(m, df_trn); fi

In [ ]:
def plot_fi(fi): return fi.plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)

In [ ]:
plot_fi(fi);

In [ ]:
df_train = create_train()